In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import os
import glob

In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [3]:
split = ['train[:70%]', 'train[70%:]']

trainDataset, testDataset = tfds.load(name='imdb_reviews', split=split, as_supervised=True)

In [4]:
BUFFER_SIZE = 10000
BATCH_SIZE = 128

train_dataset = trainDataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = testDataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [5]:
VOCAB_SIZE = 1000
encoder = tf.keras.layers.TextVectorization(
    max_tokens=VOCAB_SIZE)

encoder.adapt(train_dataset.map(lambda text, label: text))

In [6]:

SA_RNN = tf.keras.models.Sequential([
    encoder,
    tf.keras.layers.Embedding(input_dim=len(encoder.get_vocabulary()), output_dim = 128, mask_zero = True),
    tf.keras.layers.LSTM(128, return_sequences=True),
    tf.keras.layers.GRU(128),
    tf.keras.layers.Dense(64,activation='relu'),
    tf.keras.layers.Dense(1)
])

In [17]:
SA_RNN.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), optimizer=tf.keras.optimizers.Adam(0.001), metrics=['accuracy'])

In [18]:
SA_RNN.fit(train_dataset, epochs = 10)

Epoch 1/10
137/137 [==============================] - 21s 115ms/step - loss: 0.5532 - accuracy: 0.7149
Epoch 2/10
137/137 [==============================] - 16s 113ms/step - loss: 0.4116 - accuracy: 0.8083
Epoch 3/10
137/137 [==============================] - 16s 114ms/step - loss: 0.3427 - accuracy: 0.8554
Epoch 4/10
137/137 [==============================] - 16s 114ms/step - loss: 0.3373 - accuracy: 0.8566
Epoch 5/10
137/137 [==============================] - 16s 115ms/step - loss: 0.4021 - accuracy: 0.8149
Epoch 6/10
137/137 [==============================] - 16s 114ms/step - loss: 0.3355 - accuracy: 0.8538
Epoch 7/10
137/137 [==============================] - 16s 114ms/step - loss: 0.3144 - accuracy: 0.8634
Epoch 8/10
137/137 [==============================] - 16s 114ms/step - loss: 0.2839 - accuracy: 0.8806
Epoch 9/10
137/137 [==============================] - 16s 115ms/step - loss: 0.2884 - accuracy: 0.8802
Epoch 10/10
137/137 [==============================] - 16s 119ms/step - l

In [22]:
review = 'The Hottie and the Nottie is a crass, predictable, and ineptly staged gross-out comedy that serves little purpose beyond existing as another monument to Paris Hiltons vanity'

prediction = SA_RNN.predict(np.array([review]))

if prediction >= 0.0:
    print("This review is positive", prediction)
else:
    print("This review is negetive", prediction)


1/1 [==============================] - 0s 19ms/step
This review is negetive [[-0.5577238]]
